### CPU 환경

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:            3
CPU MHz:             2000.166
BogoMIPS:            4000.33
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            39424K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_si

### GPU 환경

In [ ]:
!nvidia-smi

Sat Aug  7 10:00:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### RAM 용량 확인

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            25G        580M         22G        1.1M        2.3G         24G
Swap:            0B          0B          0B


### HDD 용량 확인

In [ ]:
# 디스크 용량 확인
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         148G   38G  110G  26% /
tmpfs            64M     0   64M   0% /dev
tmpfs            13G     0   13G   0% /sys/fs/cgroup
shm              13G     0   13G   0% /dev/shm
tmpfs            13G   36K   13G   1% /var/colab
/dev/sda1       154G   41G  114G  27% /opt/bin
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware


### OS 환경

In [ ]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [ ]:
! pip3 install torchinfo
! pip3 install explain

     |████████████████████████████████| 57 kB 5.4 MB/s 
  Created wheel for explain: filename=explain-0.1.1-py3-none-any.whl size=2640 sha256=9b1f9cb39d5d5174ef2a83e42bdb46790f7ca04cc084217a22eb96b85e5caa15
  Stored in directory: /root/.cache/pip/wheels/5c/78/f0/c31eacc637c80b4eeee46348a804f46b2dc37dc242793fa89a
  Created wheel for sh: filename=sh-1.9-py3-none-any.whl size=19692 sha256=2eaac9eccb4668345f2de24e3234698e343881d5a81c9c89b317557c0f2a19e1
  Stored in directory: /root/.cache/pip/wheels/41/ab/45/b9eb2c547c2c9f603022d2c9d759941a0486b92ae91dd11142
Successfully built explain sh
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pip-tools 4.5.1 requires click>=7, but you have click 3.1 which is incompatible.
flask

## 패키지 불러오기

In [ ]:
import os
import os.path as osp
import sys
from glob import glob

import datetime
import time
import pytz

import pandas as pd
import numpy as np
from tqdm import tqdm

import sklearn

import torch
import torchvision
from torchinfo import summary

import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
import torchvision.models as models

import explain

In [ ]:
print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:', np.__version__)

print('sklearn version:', sklearn.__version__)
print('torch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)

python version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
pandas version: 1.1.5
numpy version: 1.19.5
sklearn version: 0.22.2.post1
torch version: 1.9.0+cu102
torchvision version: 0.10.0+cu102


In [ ]:
from google.colab import drive

drive.mount("/content/drive")
base_dir = "/content/drive/My Drive/Colab Notebooks/"

Mounted at /content/drive


## 모델 파라미터 (Parameter)

In [ ]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 128
IMG_HEIGHT = 128

# 배치 사이즈 임의 지정
batch_size = 16

# 데이터를 읽어올 경로를 지정함.
data_dir = base_dir + '/plz/pig/'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = base_dir + "QC_pig/classification/checkpoint/"

# 모델 이름
Model_name = "resnext50_32x4d_pig2"

# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

## Train, Validation, Test 데이터 셋 확인

In [ ]:
label=['1+','1','2']

for set_ in ['train','val','test']:
    set_size=len(glob(osp.join(data_dir, set_, '*', '*'), recursive=True))
    if set_=='val':
        print(f"{set_+'idation'} set size: {set_size}\n")
    else:
        print(f"{set_} set size: {set_size}\n")
    
    for i in range(3):
        size=len(glob(osp.join(data_dir, set_, str(i), '*'), recursive=True))
        print(f"The number of Class {label[i]}: {size} / {set_size} = {size/set_size :.2f}")
    print('-'*45)
    print()

train set size: 6816

The number of Class 1+: 2407 / 6816 = 0.35
The number of Class 1: 2429 / 6816 = 0.36
The number of Class 2: 1980 / 6816 = 0.29
---------------------------------------------

validation set size: 852

The number of Class 1+: 301 / 852 = 0.35
The number of Class 1: 303 / 852 = 0.36
The number of Class 2: 248 / 852 = 0.29
---------------------------------------------

test set size: 853

The number of Class 1+: 301 / 853 = 0.35
The number of Class 1: 304 / 853 = 0.36
The number of Class 2: 248 / 853 = 0.29
---------------------------------------------



## 데이터 셋 (Dataset) 불러오기

In [ ]:
# 이미지 transformation
test_compose=transforms.Compose([
    transforms.Resize((IMG_HEIGHT,IMG_WIDTH)),
    transforms.ToTensor()
])

# Test set을 읽어옴
test_dataset=datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_compose)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)

### 모델 불러오기 (Load)

In [ ]:
# Model: ResNeXt50_32x4d
model=models.resnext50_32x4d(pretrained=False)
# 최종 Label 의 개수 = 5 (1++, 1+, 1, 2, 3)
model.fc=nn.Linear(model.fc.in_features, 5)

# 학습된 모델 가중치 불러오기
#drive.mount("/content/drive")
#ckpt_dir = "/content/drive/My Drive/Colab Notebooks/QC_pig/classification/checkpoint/"
#print(ckpt_dir + Model_name )
checkpoint=torch.load(ckpt_dir + Model_name + '.pht', map_location=device)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

### 모델 학습 전, ImageNet 데이터로 Pretrain 된 기본 모델 불러오기 (Load)

In [ ]:
# model : ResNeXt50_32x4d
base_model=models.resnext50_32x4d(pretrained=True)
base_model.fc=nn.Linear(base_model.fc.in_features, 5)

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


## 모델 확인 (Summary)

In [ ]:
_ = summary(model, (4, 3, 512, 512))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


---

## 테스트 (Test) 수행

### 1. ImageNet 데이터로 Pretrain 된 기본 모델 성능 평가

### 테스트 이미지 평가 (Predict)

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-08-07 19:11:42


In [ ]:
since=time.time()

In [ ]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
base_model=base_model.to(device)
# 네트워크를 evaluation 용으로 선언
base_model=base_model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=base_model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

100%|██████████| 54/54 [08:27<00:00,  9.41s/it]


In [ ]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 8m 49s


In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-08-07 19:20:39


## Accuracy

In [ ]:
!pip3 uninstall explain

Found existing installation: explain 0.1.1
Uninstalling explain-0.1.1:
  Would remove:
    /usr/local/bin/explain
    /usr/local/lib/python3.7/dist-packages/explain-0.1.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/explain/*
Proceed (y/n)? y
  Successfully uninstalled explain-0.1.1


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

class ShowResult:
    def __init__(self, true_tot_labels, pred_tot_labels):
        self.true_tot_labels=true_tot_labels
        self.pred_tot_labels=pred_tot_labels
        #self.label=['1++','1+','1','2','3']
        self.label=['1+','1','2']
        
        self.multi_label_confusion_mat=multilabel_confusion_matrix(self.true_tot_labels, self.pred_tot_labels)
        self.total_num=len(true_tot_labels)
        
    def per_class_confusion_mat(self, array, label):
        index=pd.MultiIndex.from_arrays([ ['True','True'], [f'Non {label}', label] ])
        columns=pd.MultiIndex.from_arrays([ ['Pred','Pred'], [f'Non {label}', label] ])
        
        cf_mat=pd.DataFrame(array, index=index, columns=columns)
        acc=100*np.diag(cf_mat).sum()/self.total_num
        acc=round(acc, 3)        
        
        print(f'#-- Confusion Matrix for class {label}\n')
        print(cf_mat)
        print(f"\nAccuracy for class {label} : {acc}")
        print('-'*35)
        print()
        
        return acc
        
        
        
    def show_result(self):
        cf_mat=pd.crosstab(pd.Series(self.true_tot_labels), pd.Series(self.pred_tot_labels),
                               rownames=['True'], colnames=['Predicted'], margins=True)
        #cf_mat=cf_mat.rename(index={0:'1++',1:'1+',2:'1',3:'2',4:'3'},
        #              columns={0:'1++',1:'1+',2:'1',3:'2',4:'3'})
        cf_mat=cf_mat.rename(index={0:'1+',1:'1',2:'2'},
                      columns={0:'1+',1:'1',2:'2'})

        print(cf_mat)
        print()
        print()       
        
        self.total_acc=[]
        for i, label in enumerate(self.label):
            array=self.multi_label_confusion_mat[i]
            acc=self.per_class_confusion_mat(array, label)
            self.total_acc.append(acc)
            
        print(f"#-- Final Average Accuracy")
        print(f"( {self.total_acc[0]} + {self.total_acc[1]} + {self.total_acc[2]}  ) / 3= {np.mean(self.total_acc) :.3f}")

In [ ]:
result = ShowResult(tot_labels, tot_pred_labels)

In [ ]:
result.show_result()

Predicted  1+   1    2   3   4  All
True                               
1+          1  11  278   2   9  301
1           0  19  277   6   2  304
2           2  10  228   4   4  248
All         3  40  783  12  15  853


#-- Confusion Matrix for class 1+

              Pred   
            Non 1+ 1+
True Non 1+    550  2
     1+        300  1

Accuracy for class 1+ : 64.596
-----------------------------------

#-- Confusion Matrix for class 1

            Pred    
           Non 1   1
True Non 1   528  21
     1       285  19

Accuracy for class 1 : 64.127
-----------------------------------

#-- Confusion Matrix for class 2

            Pred     
           Non 2    2
True Non 2    50  555
     2        20  228

Accuracy for class 2 : 32.591
-----------------------------------

#-- Final Average Accuracy
( 64.596 + 64.127 + 32.591  ) / 3= 53.771


---

### 2. 기본 모델을 Fine Tuning (학습) 한 모델 성능 평가 (Transfer Learning)

### 테스트 이미지 평가 (Predict)

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-08-07 19:29:09


In [ ]:
since=time.time()

In [ ]:
tot_labels=[]  # 전체 정답 Label 을 저장할 리스트
tot_pred_labels=[]  # 전체 예측 Label 을 저장할 리스트

# 모델을 device에 올림
model=model.to(device)
# 네트워크를 evaluation 용으로 선언
model=model.eval()

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):

        images=images.to(device)
        labels=labels.to(device)

        # Model Prediction (forward)
        output=model(images)
        _, output_index = torch.max(output, 1)    

        tot_labels.extend(list(labels.numpy()))
        tot_pred_labels.extend(list(output_index.view(-1).numpy()))

100%|██████████| 54/54 [00:45<00:00,  1.19it/s]


In [ ]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 0m 50s


In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-08-07 19:30:02


## Accuracy

In [ ]:
result=ShowResult(tot_labels, tot_pred_labels)

In [ ]:
result.show_result()

Predicted   1+    1    2  All
True                         
1+         220   42   39  301
1           34  223   47  304
2           31   54  163  248
All        285  319  249  853


#-- Confusion Matrix for class 1+

              Pred     
            Non 1+   1+
True Non 1+    487   65
     1+         81  220

Accuracy for class 1+ : 82.884
-----------------------------------

#-- Confusion Matrix for class 1

            Pred     
           Non 1    1
True Non 1   453   96
     1        81  223

Accuracy for class 1 : 79.25
-----------------------------------

#-- Confusion Matrix for class 2

            Pred     
           Non 2    2
True Non 2   519   86
     2        85  163

Accuracy for class 2 : 79.953
-----------------------------------

#-- Final Average Accuracy
( 82.884 + 79.25 + 79.953  ) / 3= 80.696
